In [37]:
import os
import json
import datetime as dt
import pandas as pd
import numpy as np
import sys
import requests
import datetime as dt

In [38]:
##get trailhead hotspot list

trailheadHotspots = pd.DataFrame()
trailheadHotspots = pd.read_csv('https://raw.githubusercontent.com/Tanag3r/trailheadDirect_birds/main/trailheadHotspots_Y2022_M1_D6.csv')

In [39]:
ebird_token = 'j6c7l80ga2ib'

##be nice to eBird, get a filtered list from all trailhead hotspots for testing
##the code below is for testing, interchange stopNames as needed
THfilters = ['HighSchool','EastSunsetWay']
THfiltersBool = trailheadHotspots.StopName.isin(THfilters)
selectHotspots = pd.DataFrame(trailheadHotspots[THfiltersBool])

In [41]:

##get recent notable observations

notableObs_current = []

##Really, really need to switch out this loop for a function
for locId in trailheadHotspots.itertuples():
##for locId in selectHotspots.itertuples():
    ebird_url = 'https://api.ebird.org/v2/data/obs/geo/recent/notable?'
    ebird_authHeader = {'X-eBirdApiToken': ebird_token}
    ebird_params = {
        'lat': str(locId.lat),
        'lng': str(locId.lng),
        'dist': str(2),
        'back': str(30),
        'detail': 'simple',
        'fmt': 'json'
    }
    ebird_request = requests.get(ebird_url,params=ebird_params,headers=ebird_authHeader)
    if ebird_request.status_code == requests.codes.ok:
        ebird_response = pd.DataFrame(ebird_request.json())
        ebird_response['StopName'] = locId.StopName
        notableObs_current.append(ebird_response)
    ebird_request.raise_for_status()

In [42]:
all_notableObs_current = pd.concat(notableObs_current)
##sanity check
all_notableObs_current.head(25)

,StopName,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId
0,EastSunsetWay,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,EastSunsetWay,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,EastSunsetWay,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,HighSchool,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,HighSchool,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,HighSchool,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395


##get historical data on all notable observations in a +/- 30 day window

date_range = []

today = dt.date.today()
todayBack = today - dt.timedelta(30)
todayOut = today + dt.timedelta(30)

for i in range(int(today.year-1),int(today.year-4)):
    days = pd.date_range(
        start=dt.datetime(i,int(todayBack.month),int(todayBack.day)),
        end=dt.datetime(i,int(todayOut.month),int(todayOut.day))
    )
    date_range.append(days)

In [81]:
date_list = []
for sublist in date_range:
    for item in sublist:
        date_list.append(item)

In [109]:
date_range = []

today = dt.date.today()
todayBack = today - dt.timedelta(30)
todayOut = today + dt.timedelta(30)
yearStart = today.year-1
yearStop = today.year-4
monthBack = todayBack.month
monthOut = todayOut.month

for i in range(yearStop,yearStart):
    days = pd.date_range(
        start=dt.date(i,monthBack,1),
        end=dt.date(i,monthOut,1)
    )
    date_range.append(days)


In [108]:
date_list = []
for sublist in date_range:
    for item in sublist:
        date_list.append(item)

print(date_range)

[DatetimeIndex([], dtype='datetime64[ns]', freq='D'), DatetimeIndex([], dtype='datetime64[ns]', freq='D'), DatetimeIndex([], dtype='datetime64[ns]', freq='D')]
